In [4]:
import time
import socket
from operator import itemgetter

In [3]:
class ClientError(Exception):
    pass

In [34]:
class Client:
    
    def __init__(self,host,port,timeout=None):
        self.host=host
        self.port=port
        self.timeout=timeout
        
        
        
    def sending(self, cmd_str):
        with socket.create_connection((self.addr, self.port), self.timeout) as sock:
            sock.sendall(cmd_str.encode("utf8"))
            data = sock.recv(1024)
            return data.decode('utf-8')
    
    def put(self, metric_name, metric_value, timestamp=None): 
        
        timestamp=timestamp or int(time.time())
        
        message = f'put {metric_name} {metric_value} {timestamp}\n'
        
        try:
            recv_data= self.sending(message)
            
                       
            if recv_data[0:3] != 'ok\n':
                raise ClientError
            
        except:
            raise ClientError
        
        
    def get(self,metric_name):
        
        k={}
        message = f'get {metric_name}\n'
        recv_data=self.sending(message)
        if recv_data == "ok\n\n":
                return {}
               
        else:
            for x in recv_data.rstrip().split('\n')[1:]:
                j=x.split(' ')
                key=j[0]
                k.setdefault(key,[])
                k[key].append((int(j[2]),float(j[1])))
         
            
            for val in k.values():
                val.sort(key=itemgetter(0))
                
    
        return k if metric_name == '*' else {metric_name : k[metric_name]}
            
          
            
          
    
    
    
    
    